In [ ]:
###드라이브에 사전에 데이터 파일 저장
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
! pip install pycaret


     |████████████████████████████████| 256kB 9.5MB/s 
     |████████████████████████████████| 6.8MB 12.8MB/s 
     |████████████████████████████████| 1.8MB 30.7MB/s 
     |████████████████████████████████| 276kB 39.4MB/s 
     |████████████████████████████████| 102kB 7.1MB/s 
     |████████████████████████████████| 157.5MB 79kB/s 
     |████████████████████████████████| 65.8MB 66kB/s 
     |████████████████████████████████| 245kB 46.6MB/s 
     |████████████████████████████████| 14.2MB 32.1MB/s 
     |████████████████████████████████| 1.6MB 38.1MB/s 
     |████████████████████████████████| 174kB 40.7MB/s 
     |████████████████████████████████| 2.1MB 41.0MB/s 
     |████████████████████████████████| 604kB 43.7MB/s 
     |████████████████████████████████| 3.1MB 33.0MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
     |████████████████████████████████| 163kB 44.0MB/s 


In [ ]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
filename1='/content/drive/MyDrive/LG대회/train_err_data.csv' #각자 드라이브에 파일저장위치 file1~2에 입력
filename2='/content/drive/MyDrive/LG대회/train_problem_data.csv'
filename3='/content/drive/MyDrive/LG대회/train_quality_data.csv'
train_err=pd.read_csv(filename1)
print(train_err[:5])

   user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0


In [ ]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((15000,42))
for person_idx, err in tqdm(id_error):
    # person_idx - 10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - 10000,err - 1] += 1

train_prob = pd.read_csv(filename2)
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 

train = pd.DataFrame(data=error)
train['problem'] = problem
del error, problem

clf = setup(data = train, target = 'problem') 

,Description,Value
0,session_id,8463
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 43)"
5,Missing Values,False
6,Numeric Features,42
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7826,0.8112,0.4944,0.7688,0.6015,0.4616,0.4830,10.803
lightgbm,Light Gradient Boosting Machine,0.7813,0.8050,0.5062,0.7546,0.6058,0.4625,0.4803,0.334
gbc,Gradient Boosting Classifier,0.7783,0.8064,0.4623,0.7813,0.5806,0.4431,0.4714,2.036
et,Extra Trees Classifier,0.7768,0.8008,0.4818,0.7585,0.5891,0.4462,0.4680,1.692
rf,Random Forest Classifier,0.7761,0.7991,0.4752,0.7620,0.5848,0.4426,0.4661,1.887
ada,Ada Boost Classifier,0.7725,0.7935,0.4646,0.7573,0.5754,0.4320,0.4563,0.568
xgboost,Extreme Gradient Boosting,0.7711,0.7884,0.5151,0.7169,0.5991,0.4449,0.4570,4.473
lr,Logistic Regression,0.7523,0.7325,0.3697,0.7620,0.4970,0.3583,0.3998,1.755
qda,Quadratic Discriminant Analysis,0.7507,0.7371,0.3903,0.7352,0.5097,0.3630,0.3959,0.059
lda,Linear Discriminant Analysis,0.7463,0.7395,0.3209,0.7916,0.4565,0.3276,0.3846,0.096


In [ ]:
blended=blend_models(estimator_list=best_5, fold=5, method='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7790,0.8114,0.4628,0.7840,0.5820,0.4451,0.4736
1,0.7890,0.8203,0.4943,0.7931,0.6090,0.4750,0.4999
2,0.7962,0.8266,0.4964,0.8180,0.6179,0.4900,0.5184
3,0.7643,0.7868,0.4476,0.7393,0.5576,0.4099,0.4339
4,0.7808,0.8107,0.4935,0.7627,0.5993,0.4579,0.4784
Mean,0.7819,0.8112,0.4789,0.7794,0.5932,0.4556,0.4808
SD,0.0107,0.0135,0.0200,0.0268,0.0214,0.0274,0.0284


In [ ]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.78,0.7996,0.4693,0.7915,0.5892,0.4521,0.4809


In [ ]:
final_model = finalize_model(blended)

In [ ]:
test_err  = pd.read_csv('/content/drive/MyDrive/LG대회/test_err_data.csv')
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx, err in tqdm(id_error):
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
test = pd.DataFrame(data=test_x)

100%|██████████| 16532648/16532648 [00:47<00:00, 346731.27it/s]


In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])

In [ ]:
sample_submssion = pd.read_csv('/content/drive/MyDrive/LG대회/sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("pycaret.csv", index = False)